# Exploratory Analysis Phishing_Email.csv Group 3

In [3]:
# Potentionally helpful libraries:
# pandas, matplotlib, seaborn, nltk, scikit-learn, gensim, pyLDAvis

import pandas as pd
import re

In [4]:
def make_df():
    # Reading in the data
    df = pd.read_csv("Phishing_Email.csv")
    # Dropping the unecessary index row
    df = df.drop(df.columns[0], axis=1)
    
    # Removing Nulls
    # Check for null values in the "Email Text" column
    null_mask = df["Email Text"].isnull()

    # If there are null values, drop the corresponding rows
    if null_mask.any():
        df = df.drop(df[null_mask].index)
        
    #Remove 'empty' emails from the dataframe
    df_cleaned = df[df['Email Text'].str.lower() != 'empty']
    df = df_cleaned
    
    return df

In [53]:
# # Reading in the data
# df_phish = pd.read_csv("Phishing_Email.csv")
# # Dropping the unecessary index row
# df_phish = df_phish.drop(df_phish.columns[0], axis=1)
# df_phish[0:10]

,Email Text,Email Type
0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email
1,the other side of * galicismos * * galicismo *...,Safe Email
2,re : equistar deal tickets are you still avail...,Safe Email
3,\r\nHello I am your hot lil horny toy.\r\n ...,Phishing Email
4,software at incredibly low prices ( 86 % lower...,Phishing Email
5,global risk management operations sally congra...,Safe Email
6,"On Sun, Aug 11, 2002 at 11:17:47AM +0100, wint...",Safe Email
7,"entourage , stockmogul newsletter ralph velez ...",Phishing Email
8,"we owe you lots of money dear applicant , afte...",Phishing Email
9,re : coastal deal - with exxon participation u...,Safe Email


## Data Cleaning

In [54]:
# NullCount = df_phish["Email Text"].isnull().sum()
# print("Nulls:", NullCount)
# # Check for null values in the "Email Text" column
# null_mask = df_phish["Email Text"].isnull()

# # If there are null values, drop the corresponding rows
# if null_mask.any():
#     df_phish = df_phish.drop(df_phish[null_mask].index)
# # Checking if null values were dropped
# NullCount = df_phish["Email Text"].isnull().sum()
# print("Nulls:", NullCount)

# # Visual Inspection
# print(df_phish.head(10))
# print(df_phish.tail(20))


Nulls: 16
Nulls: 0
                                          Email Text      Email Type
0  re : 6 . 1100 , disc : uniformitarianism , re ...      Safe Email
1  the other side of * galicismos * * galicismo *...      Safe Email
2  re : equistar deal tickets are you still avail...      Safe Email
3  \r\nHello I am your hot lil horny toy.\r\n    ...  Phishing Email
4  software at incredibly low prices ( 86 % lower...  Phishing Email
5  global risk management operations sally congra...      Safe Email
6  On Sun, Aug 11, 2002 at 11:17:47AM +0100, wint...      Safe Email
7  entourage , stockmogul newsletter ralph velez ...  Phishing Email
8  we owe you lots of money dear applicant , afte...  Phishing Email
9  re : coastal deal - with exxon participation u...      Safe Email
                                              Email Text      Email Type
18630  s . hattori dies this is the english version o...      Safe Email
18631  Update of /cvsroot/spamassassin/spamassassin\r...      Safe Email
186

In [55]:
# # Check if each "Email Text" is type string
# print(df_phish.shape)
# # Check if each "Email Text" is type string
# is_string = df_phish["Email Text"].apply(lambda x: isinstance(x, str))
# print("Number of non-string values in 'Email Text':", (~is_string).sum())
# print(df_phish.shape)

(18634, 2)
Number of non-string values in 'Email Text': 0
(18634, 2)


In [56]:
#Get count of 'empty' emails
# empty_email_count = df_phish['Email Text'].str.lower().eq('empty').sum()
# print(f"Number of 'empty' emails: {empty_email_count}")
# #Remove those 'empty' emails from the dataframe
# df_phish_cleaned = df_phish[df_phish['Email Text'].str.lower() != 'empty']
# df_phish_cleaned.shape
# empty_email_count = df_phish_cleaned['Email Text'].str.lower().eq('empty').sum()
# print(f"Number of 'empty' emails: {empty_email_count}")
# df_phish = df_phish_cleaned

Number of 'empty' emails: 533
Number of 'empty' emails: 0


## Feature Extraction

### Emails with "re:"

In [5]:
df_phish = make_df()

print(df_phish.tail(5))

def countNaN(df):
    return (df["Email Text"] == "NaN").sum()

print(countNaN(df_phish))

                                              Email Text      Email Type
18644  \r\nRick Moen  a Ã©crit:> > I'm confused. I th...      Safe Email
18645  date a lonely housewife always wanted to date ...  Phishing Email
18646  request submitted : access request for anita ....      Safe Email
18647  re : important - prc mtg hi dorn & john , as y...      Safe Email
18648  press clippings - letter on californian utilit...      Safe Email
0


In [7]:
print(df_phish.tail(20))
# Adding "Is_Response" feature
df_phish["Is_Response"] = df_phish["Email Text"].str.contains("re :")
print(df_phish.tail(20))

                                              Email Text      Email Type
18627  crying mlcros 0 ft , symanntec , macromedia , ...  Phishing Email
18628  Has anybody on the list installed/used this di...      Safe Email
18629  re : 6 . 923 , sum : chomsky on letting the th...      Safe Email
18630  s . hattori dies this is the english version o...      Safe Email
18631  Update of /cvsroot/spamassassin/spamassassin\r...      Safe Email
18632  weekly deal report this is the weekly report c...      Safe Email
18633  This article from NYTimes.com \r\nhas been sen...      Safe Email
18634  congratulations you have won ! ! ! pls contact...  Phishing Email
18635  On Fri, 4 Oct 2002, Mark Derricutt wrote:> Any...      Safe Email
18636  prior month misnomination daren - - - - for 4 ...      Safe Email
18638  strong buy alert : monthly newsletter topstock...  Phishing Email
18639  out of office i have an appointment today and ...      Safe Email
18640  list of english taboo words does anyone have

In [58]:
df_phish["Is_Response"].value_counts(normalize=True)

Is_Response
False    0.866637
True     0.133363
Name: proportion, dtype: float64

### Emails with links

In [9]:
# This is causing columns to turn into NaN values and losing data.

print(df_phish.tail(5))
# Adding "Has_WebLink" column
df_phish["Has_WebLink"] = df_phish["Email Text"].str.contains('(https?://|www.)')

print(df_phish.tail(5))
    

                                              Email Text      Email Type  \
18644  \r\nRick Moen  a Ã©crit:> > I'm confused. I th...      Safe Email   
18645  date a lonely housewife always wanted to date ...  Phishing Email   
18646  request submitted : access request for anita ....      Safe Email   
18647  re : important - prc mtg hi dorn & john , as y...      Safe Email   
18648  press clippings - letter on californian utilit...      Safe Email   

       Is_Response  Has_WebLink  
18644        False         True  
18645         True        False  
18646        False        False  
18647         True        False  
18648        False        False  


C:\Users\jakos\Documents\Visual Studio Code\data\tmp\ipykernel_16076\4288188749.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_phish["Has_WebLink"] = df_phish["Email Text"].str.contains('(https?://|www.)')


                                              Email Text      Email Type  \
18644  \r\nRick Moen  a Ã©crit:> > I'm confused. I th...      Safe Email   
18645  date a lonely housewife always wanted to date ...  Phishing Email   
18646  request submitted : access request for anita ....      Safe Email   
18647  re : important - prc mtg hi dorn & john , as y...      Safe Email   
18648  press clippings - letter on californian utilit...      Safe Email   

       Is_Response  Has_WebLink  
18644        False         True  
18645         True         True  
18646        False        False  
18647         True        False  
18648        False        False  


In [60]:
df_phish["Has_WebLink"].value_counts(normalize=True)

Has_WebLink
False    0.769184
True     0.230816
Name: proportion, dtype: float64

### Email Length

In [62]:
print(df_phish.tail(5))

# Creating the Email Length Column
df_phish["Email_Length"] = df_phish["Email Text"].str.len()
    
# Verifying it worked
print(df_phish.tail(5))


                                              Email Text      Email Type  \
18644  \r\nRick Moen  a Ã©crit:> > I'm confused. I th...      Safe Email   
18645  date a lonely housewife always wanted to date ...  Phishing Email   
18646  request submitted : access request for anita ....      Safe Email   
18647  re : important - prc mtg hi dorn & john , as y...      Safe Email   
18648  press clippings - letter on californian utilit...      Safe Email   

       Is_Response  Has_WebLink  Email_Length  
18644        False         True          4510  
18645         True        False           237  
18646        False        False           477  
18647         True        False          1214  
18648        False        False           213  
                                              Email Text      Email Type  \
18644  \r\nRick Moen  a Ã©crit:> > I'm confused. I th...      Safe Email   
18645  date a lonely housewife always wanted to date ...  Phishing Email   
18646  request submitted : 

In [63]:
print(df_phish.tail(10))

df_phish["Hypen_Count"] = df_phish["Email Text"].str.count(r"-")
df_phish["Pound_Count"] = df_phish["Email Text"].str.count(r"#")
df_phish["At_Count"] = df_phish["Email Text"].str.count(r"@")

print(df_phish.tail(10))

                                              Email Text      Email Type  \
18638  strong buy alert : monthly newsletter topstock...  Phishing Email   
18639  out of office i have an appointment today and ...      Safe Email   
18640  list of english taboo words does anyone have a...      Safe Email   
18641  interview with the enron research group good m...      Safe Email   
18642  URL: http://diveintomark.org/archives/2002/09/...      Safe Email   
18644  \r\nRick Moen  a Ã©crit:> > I'm confused. I th...      Safe Email   
18645  date a lonely housewife always wanted to date ...  Phishing Email   
18646  request submitted : access request for anita ....      Safe Email   
18647  re : important - prc mtg hi dorn & john , as y...      Safe Email   
18648  press clippings - letter on californian utilit...      Safe Email   

       Is_Response  Has_WebLink  Email_Length  
18638        False        False          7999  
18639        False        False           256  
18640        False 

In [9]:
print(df_phish.tail(10))

df_phish["Exclamation_Count"] = df_phish["Email Text"].str.count(r"!")
df_phish["Question_Count"] = df_phish["Email Text"].str.count(r"\?")
df_phish["Period_Count"] = df_phish["Email Text"].str.count(r"\.")

print(df_phish.tail(10))

                                              Email Text      Email Type  \
18638  strong buy alert : monthly newsletter topstock...  Phishing Email   
18639  out of office i have an appointment today and ...      Safe Email   
18640  list of english taboo words does anyone have a...      Safe Email   
18641  interview with the enron research group good m...      Safe Email   
18642  URL: http://diveintomark.org/archives/2002/09/...      Safe Email   
18644  \r\nRick Moen  a Ã©crit:> > I'm confused. I th...      Safe Email   
18645  date a lonely housewife always wanted to date ...  Phishing Email   
18646  request submitted : access request for anita ....      Safe Email   
18647  re : important - prc mtg hi dorn & john , as y...      Safe Email   
18648  press clippings - letter on californian utilit...      Safe Email   

       Exclamation_Count  Question_Count  Period_Count  
18638                  1               0            57  
18639                  0               0         

### Emails with All-Caps characters over a certain percent threshold (8%) to determine safe or phishing emails

In [ ]:
#writing a function that calculates the percent of upper and lowercase characters in an email

def percent_of_all_caps(text):
    if not text or not isinstance(text, str):
        return 0
    #getting rid of special characters from check for all-caps
    alphanumeric_text = re.sub(r'[^A-Za-z0-9]','',text)
    num_all_caps =  sum(1 for c in alphanumeric_text if c.isupper())
    num_total_characters = len(alphanumeric_text)
    #prevent divide by 0
    if(num_total_characters == 0):
        return 0
    percent_all_caps = (num_all_caps/num_total_characters)*100
    return percent_all_caps

#creating a new column in dataframe for percentage of capitalization and marking emails as safe or phishing
df_phish['Capitalization_Percent'] = df_phish['Email Text'].apply(percent_of_all_caps)

# I dont want to assume a percentage value that would be indicative of a phishing email or not just based on capitalization percentage.
# We should let the model determine that or make a graph showing at different percentages, what proportion of emails were correctly identified as phishing.
# df_phish['Is Phishing'] = df_phish['Capitalization_Percent'] > 8    #initially had as 55, looked at exported cleaned data to adjust to 8

#export in a CSV file to better compare my results on phishing classification vs. actual results
# temp_df = df_phish[['Email Text', 'Email Type', 'Capitalization_Percent', 'Is Phishing']]
#temp_df.to_csv('test.csv', index=False)
#print(temp_df)